
**<h1>Content Based Rcoemmender system for Auctions </h1>**


<h2>1- Import needed librairies and datasets</h2>

In [81]:
import pandas as pd 
import numpy as np 

#Import dataset twice because the first one will be changed 
df2=pd.read_csv('C:/datasets/auctions.csv')
df3=pd.read_csv('C:/datasets/auctions.csv')


<h2>2- Dataset overview</h2>

In [82]:
df2.head(10)

__v                       _id  active                 category  \
0    0  407f1f77bcf86cd79943901a   False      jewelry and watches   
1    0  407f1f77bcf86cd79943901b   False  antiques and primitives   
2    0  407f1f77bcf86cd79943901c    True       phones & computers   
3    0  407f1f77bcf86cd79943901d    True  antiques and primitives   
4    0  407f1f77bcf86cd79943901e    True              electronics   
5    0  407f1f77bcf86cd79943901f    True              electronics   
6    0  6241a586dd804b5ba907bf58    True              automotives   
7    0  6241a586dd804b5ba907bf59    True    clothing and wellness   
8    0  6241a586dd804b5ba907bf5c    True      jewelry and watches   
9    0  6241a586dd804b5ba907bf5d    True             collectibles   

                closingDate  currentPrice  \
0  2022-04-08T10:49:00.061Z         10057   
1  2022-04-15T14:36:37.854Z          4093   
2  2022-03-27T12:42:56.383Z          1100   
3  2022-03-25T00:05:27.070Z          1788   
4  2022-03-26T00:05:59.831Z          1854   
5  2022-03-27T01:26:47.877Z          1543   
6  2022-03-28T08:13:15.503Z          1490   
7  2022-03-28T12:05:54.793Z          1710   
8  2022-03-28T15:40:03.023Z          1893   
9  2022-03-29T15:39:38.047Z          4815   

                                         description  \
0  Carbonite web goalkeeper gloves are ergonomica...   
1  The Football Is Good For Training And Recreati...   
2  Ergonomic executive chair upholstered in bonde...   
3  The Apollotech B340 is an affordable wireless ...   
4  The slim & simple Maple Gaming Keyboard from D...   
5  The slim & simple Maple Gaming Keyboard from D...   
6  The Nagasaki Lander is the trademarked name of...   
7  The slim & simple Maple Gaming Keyboard from D...   
8  The automobile layout consists of a front-engi...   
9  Carbonite web goalkeeper gloves are ergonomica...   

                                              images  numberOfBids  \
0  ["https://source.unsplash.com/category/technol...             7   
1  ["https://source.unsplash.com/category/technol...             5   
2  ["https://source.unsplash.com/category/technol...             1   
3  ["https://source.unsplash.com/category/technol...             6   
4  ["https://source.unsplash.com/category/technol...            13   
5  ["https://source.unsplash.com/category/technol...             6   
6  ["https://source.unsplash.com/category/technol...             3   
7  ["https://source.unsplash.com/category/technol...             7   
8  ["https://source.unsplash.com/category/technol...            16   
9  ["https://source.unsplash.com/category/technol...            14   

   preferredPrice              startingDate  startingPrice thumbnail  \
0            9460  2022-03-16T06:26:30.102Z            953       NaN   
1            4052  2022-03-14T07:25:42.586Z            130       NaN   
2            5315  2022-03-16T19:24:46.394Z            104       NaN   
3            1886  2022-03-15T04:09:11.037Z            739       NaN   
4            5535  2022-03-16T20:58:32.823Z            301       NaN   
5            6368  2022-03-15T09:44:31.958Z            668       NaN   
6            4859  2022-03-16T08:37:21.416Z            980       NaN   
7            3879  2022-03-14T13:44:09.532Z            643       NaN   
8            8759  2022-03-14T11:48:35.552Z            162       NaN   
9            6240  2022-03-15T15:57:51.542Z            679       NaN   

                       title                      user  \
0           Sleek Steel Ball  507f1f77bcf86cd79943901a   
1    Ergonomic Granite Bacon  507f1f77bcf86cd79943901b   
2  Practical Concrete Cheese  507f1f77bcf86cd79943901c   
3    Incredible Frozen Pants  507f1f77bcf86cd79943901d   
4         Rustic Metal Chair  507f1f77bcf86cd79943901e   
5   Incredible Plastic Shirt  507f1f77bcf86cd79943901f   
6   Handcrafted Rubber Mouse  507f1f77bcf86cd79943901d   
7    Incredible Wooden Table  507f1f77bcf86cd79943901d   
8     Handcrafted Metal Tuna  507f1f77bcf86c

In [83]:

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   __v             115 non-null    int64 
 1   _id             115 non-null    object
 2   active          115 non-null    bool  
 3   category        64 non-null     object
 4   closingDate     115 non-null    object
 5   currentPrice    115 non-null    int64 
 6   description     115 non-null    object
 7   images          115 non-null    object
 8   numberOfBids    115 non-null    int64 
 9   preferredPrice  115 non-null    int64 
 10  startingDate    115 non-null    object
 11  startingPrice   115 non-null    int64 
 12  thumbnail       58 non-null     object
 13  title           115 non-null    object
 14  user            115 non-null    object
 15  winningBidder   6 non-null      object
dtypes: bool(1), int64(5), object(10)
memory usage: 13.7+ KB


<h2>3- Dataset cleaning</h2>

In [84]:
df2=df2.drop(['__v'], axis=1)
df3=df3.drop(['__v'], axis=1)
df2 = df2.drop(df2[df2.active == False].index)
df3 = df3.drop(df3[df3.active == False].index)

df2 = df2.reset_index()
df3 = df3.reset_index()

In [85]:
df3.shape


(109, 16)

<h2>4- Auction title based recommender</h2>

In [86]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'....
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df2['title'] = df2['title'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['title'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(109, 61)

In [87]:
# print(tfidf_matrix)

In [88]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [89]:
# Function that takes in auction auction_id as input and outputs most similar auctions
def get_recommendations_title(auction_id, cosine_sim=cosine_sim):
    # Get the index of the auction that matches the auction_id
    idx =df2.index[df2['_id'] == auction_id][0]
    
    # Get the pairwsie similarity scores of all auctions with that auction
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the auctions based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar auctions
    sim_scores = sim_scores[1:11]

    # Get the auction indices
    auction_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar auctions
    return df3[['title','description','category','user']].iloc[auction_indices]

In [90]:
#get_recommendations_title("407f1f77bcf86cd79943901b")

<h2>5- Auction metadata based recommender</h2>

In [91]:

# from ast import literal_eval


## Parse the stringified features into their corresponding python objects
# features = ['keywords']
# for feature in features:
#     df2[feature] = df2[feature].apply(literal_eval)

In [92]:
# # Returns the list top 3 elements or entire list; whichever is more.
# def get_list(x):
#     if isinstance(x, list):
#         names = [i['name'] for i in x]
#         #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
#         if len(names) > 3:
#             names = names[:3]
#         return names

#     #Return empty list in case of missing/malformed data
#     return []

In [93]:
# # Define keywords features that are in a suitable form.


# features = [keywords']
# for feature in features:
#     df2[feature] = df2[feature].apply(get_list)

In [94]:
df2[['category', 'title', 'description']].head(3)

category                      title  \
0       phones & computers  Practical Concrete Cheese   
1  antiques and primitives    Incredible Frozen Pants   
2              electronics         Rustic Metal Chair   

                                         description  
0  Ergonomic executive chair upholstered in bonde...  
1  The Apollotech B340 is an affordable wireless ...  
2  The slim & simple Maple Gaming Keyboard from D...

In [95]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    
    if isinstance(x, list):
        
        return [str.lower(i.replace(" ", "")) for i in x]
        
    else:
        #Check if director exists. If not, return empty string
       
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [96]:

# df3["keywords"] = df2["keywords"].apply(clean_data)

In [97]:
# Apply clean_data function to your features.
features = ['category']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [98]:
def create_soup(x):
    #return ( x['category']+' '+x['title']+' '+x['keywords']+' '+x['user'])
    return ( x['category']+' '+x['title']+ ' '+x['description']+" "+x['user'])
df2['soup'] = df2.apply(create_soup, axis=1)
print(df2['soup'][0])

phones&computers Practical Concrete Cheese Ergonomic executive chair upholstered in bonded black leather and PVC padded seat and back for all-day comfort and support 507f1f77bcf86cd79943901c


In [99]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')

count_matrix = count.fit_transform(df2['soup'])


In [100]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [101]:
# Function that takes in auction auction_id as input and outputs most similar auctions
def get_recommendations_metadata(auction_id, cosine_sim=cosine_sim2):
    # Get the index of the auction that matches the auction_id
    idx =df2.index[df2['_id'] == auction_id][0]
    
    # Get the pairwsie similarity scores of all auctions with that auction
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the auctions based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar auctions
    sim_scores = sim_scores[1:11]

    # Get the auction indices
    auction_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar auctions
    return df3['_id'].iloc[auction_indices].to_json(orient = 'records')

In [102]:
# import pickle
# pickle.dump(get_recommendations_metadata, open("model.pkl", "wb"))

In [103]:
# model = pickle.load(open("./model.pkl", "rb"))
# my_personality = model('407f1f77bcf86cd79943901c')
# print('result: ', my_personality)

In [104]:
get_recommendations_metadata('6241a586dd804b5ba907bf60')

'["6241a586dd804b5ba907bf61","407f1f77bcf86cd79943901c","6241a586dd804b5ba907bf67","407f1f77bcf86cd79943901e","6241a586dd804b5ba907bf68","6241a586dd804b5ba907bf6a","407f1f77bcf86cd79943901d","407f1f77bcf86cd79943901f","6241a586dd804b5ba907bf59","6241a586dd804b5ba907bf58"]'

In [105]:
df3.head()

index                       _id  active                 category  \
0      2  407f1f77bcf86cd79943901c    True       phones & computers   
1      3  407f1f77bcf86cd79943901d    True  antiques and primitives   
2      4  407f1f77bcf86cd79943901e    True              electronics   
3      5  407f1f77bcf86cd79943901f    True              electronics   
4      6  6241a586dd804b5ba907bf58    True              automotives   

                closingDate  currentPrice  \
0  2022-03-27T12:42:56.383Z          1100   
1  2022-03-25T00:05:27.070Z          1788   
2  2022-03-26T00:05:59.831Z          1854   
3  2022-03-27T01:26:47.877Z          1543   
4  2022-03-28T08:13:15.503Z          1490   

                                         description  \
0  Ergonomic executive chair upholstered in bonde...   
1  The Apollotech B340 is an affordable wireless ...   
2  The slim & simple Maple Gaming Keyboard from D...   
3  The slim & simple Maple Gaming Keyboard from D...   
4  The Nagasaki Lander is the trademarked name of...   

                                              images  numberOfBids  \
0  ["https://source.unsplash.com/category/technol...             1   
1  ["https://source.unsplash.com/category/technol...             6   
2  ["https://source.unsplash.com/category/technol...            13   
3  ["https://source.unsplash.com/category/technol...             6   
4  ["https://source.unsplash.com/category/technol...             3   

   preferredPrice              startingDate  startingPrice thumbnail  \
0            5315  2022-03-16T19:24:46.394Z            104       NaN   
1            1886  2022-03-15T04:09:11.037Z            739       NaN   
2            5535  2022-03-16T20:58:32.823Z            301       NaN   
3            6368  2022-03-15T09:44:31.958Z            668       NaN   
4            4859  2022-03-16T08:37:21.416Z            980       NaN   

                       title                      user winningBidder  
0  Practical Concrete Cheese  507f1f77bcf86cd79943901c           NaN  
1    Incredible Frozen Pants  507f1f77bcf86cd79943901d           NaN  
2         Rustic Metal Chair  507f1f77bcf86cd79943901e           NaN  
3   Incredible Plastic Shirt  507f1f77bcf86cd79943901f           NaN  
4   Handcrafted Rubber Mouse  507f1f77bcf86cd79943901d           NaN

In [106]:
df2.head()

index                       _id  active               category  \
0      2  407f1f77bcf86cd79943901c    True       phones&computers   
1      3  407f1f77bcf86cd79943901d    True  antiquesandprimitives   
2      4  407f1f77bcf86cd79943901e    True            electronics   
3      5  407f1f77bcf86cd79943901f    True            electronics   
4      6  6241a586dd804b5ba907bf58    True            automotives   

                closingDate  currentPrice  \
0  2022-03-27T12:42:56.383Z          1100   
1  2022-03-25T00:05:27.070Z          1788   
2  2022-03-26T00:05:59.831Z          1854   
3  2022-03-27T01:26:47.877Z          1543   
4  2022-03-28T08:13:15.503Z          1490   

                                         description  \
0  Ergonomic executive chair upholstered in bonde...   
1  The Apollotech B340 is an affordable wireless ...   
2  The slim & simple Maple Gaming Keyboard from D...   
3  The slim & simple Maple Gaming Keyboard from D...   
4  The Nagasaki Lander is the trademarked name of...   

                                              images  numberOfBids  \
0  ["https://source.unsplash.com/category/technol...             1   
1  ["https://source.unsplash.com/category/technol...             6   
2  ["https://source.unsplash.com/category/technol...            13   
3  ["https://source.unsplash.com/category/technol...             6   
4  ["https://source.unsplash.com/category/technol...             3   

   preferredPrice              startingDate  startingPrice thumbnail  \
0            5315  2022-03-16T19:24:46.394Z            104       NaN   
1            1886  2022-03-15T04:09:11.037Z            739       NaN   
2            5535  2022-03-16T20:58:32.823Z            301       NaN   
3            6368  2022-03-15T09:44:31.958Z            668       NaN   
4            4859  2022-03-16T08:37:21.416Z            980       NaN   

                       title                      user winningBidder  \
0  Practical Concrete Cheese  507f1f77bcf86cd79943901c           NaN   
1    Incredible Frozen Pants  507f1f77bcf86cd79943901d           NaN   
2         Rustic Metal Chair  507f1f77bcf86cd79943901e           NaN   
3   Incredible Plastic Shirt  507f1f77bcf86cd79943901f           NaN   
4   Handcrafted Rubber Mouse  507f1f77bcf86cd79943901d           NaN   

                                                soup  
0  phones&computers Practical Concrete Cheese Erg...  
1  antiquesandprimitives Incredible Frozen Pants ...  
2  electronics Rustic Metal Chair The slim & simp...  
3  electronics Incredible Plastic Shirt The slim ...  
4  automotives Handcrafted Rubber Mouse The Nagas...